In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import copy
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
cpu = torch.device('cpu')#"cuda:0" if torch.cuda.is_available() else "cpu")
dv=torch.device('cuda')#torch.device("cuda" if torch.cuda.is_available() else "cpu")import torchmetrics
import torchmetrics
acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)

In [53]:
from load_pums import load_data,get_default_meta

In [54]:
features,tasks=get_default_meta()
N_tasks=len(tasks)

In [55]:
states=['CA','FL','GA','AL','AZ','AR','CO','CT','IL','IN','IA','KS','LA','MD','MA','MI','MN','MO','NY','NC','OH',
       'OR','PA','TX','VA','WA','WI']

In [56]:
from model_class import STL
from my_util import DM_rate,fair_loss
from copy import deepcopy

In [59]:
def trainstl_pums(year='2018',shapes=[1024,128,256,128]):
    year=str(year)
    dir_path='/yourpath/Grouping_FMTL/models/stl/pums/'
    criteria = nn.CrossEntropyLoss()
    X_train, X_val,in_tr,in_val,y_train,y_v,g_train,g_val='','','','','','','',''

    
    depth=len(shapes)
    SLs=[nn.DataParallel(STL(len(features),shapes=shapes)).to(dv) for t in range(N_tasks)]
    SL_optis=[optim.AdamW(SLs[t].parameters()) for t in range(N_tasks)]
    spaths={'path'+str(t):dir_path+year+'_stl'+str(t)+'.pt' for t in range(N_tasks)}
    j=0
    for epoch in range(5):  # loop over each state multiple times
        st_all=['CA']#deepcopy(states)
        cntr=[[],[]]
        for k in range(len(states)):
            st=np.random.choice(st_all)
            #st_all.remove(st)
            #print('State',st)
            del X_train, X_val,in_tr,in_val,y_train,y_v,g_train,g_val
            X_train, X_val,in_tr,in_val,y_train,y_v,g_train,g_val=load_data(mode='train',year=year,state_list=[st])
            xg=torch.tensor(g_train)
            i,batch=0,8192

            while(i<len(X_train)):
                # get the inputs; data is a list of [inputs, labels]
                if (i+batch)<len(X_train):
                    inputs, in_t = torch.tensor(X_train[i:i+batch]),in_tr[i:i+batch]
                    labels=[y_train[t][i:i+batch] for t in range(N_tasks)]#,y3_train[i:i+batch]]
                    #if epoch<pretrn:
                    xc=xg[i:i+batch]
                    i=i+batch 
                else:
                    inputs,in_t = torch.tensor(X_train[i:]),in_tr[i:]
                    labels=[y_train[t][i:] for t in range(N_tasks)]#,y2_train[i:]]#,y3_train[i:]]
                    #if epoch<pretrn:
                    xc=xg[i:]
                    i=len(X_train)

                # zero the parameter gradients
                for t in range(N_tasks):
                    ##training STLs
                    SL_optis[t].zero_grad()
                    out=SLs[t](inputs.to(dv).float())
                    loss_Sa=criteria(out, labels[t].to(dv))
                    loss_Sf=fair_loss(out, labels[t].to(dv),xc.to(dv))
                    #SL_E[t].append([loss_Sa,loss_Sf])
                    loss=loss_Sa+loss_Sf
                    loss.backward()
                    SL_optis[t].step()


            with torch.no_grad():

                for t in range(N_tasks): 
                    #pred_SL=SLs[t](torch.tensor(X_val).to(dv).float())
                    #SL_acc=acc(pred_SL.to(cpu),torch.tensor(y_v[t]).to(cpu))
                    #SL_eo, SL_ep=DM_rate(pred_SL.to(cpu),torch.tensor(y_v[t]).to(cpu),torch.tensor(g_val).to(cpu))
                    torch.save(SLs[t].state_dict(),spaths['path'+str(t)])
                    #print('Task',t,'  Acc:',SL_acc, 'EO:',SL_eo, 'EP:',SL_ep)         
    return SLs

In [60]:
def test_pums(year='2019',shapes=[1024,128,256,512,128]):
    if str(year)=='2019':
        tr_yr='2018'
    else:
        tr_yr='2019'
    X_test, y_test,g_test='','',''
    test_accuracy=[[] for i in range(N_tasks)]
    test_fairness=[[] for i in range(N_tasks)]
    depth=len(shapes)
    year=str(year)
    dir_path='/yourpath/Grouping_FMTL/models/stls/pums/'
    SL=nn.DataParallel(STL(len(features),shapes=shapes)).to(dv) 
    spaths={'path'+str(t):dir_path+tr_yr+'_stl'+str(t)+'.pt' for t in range(N_tasks)}

    for st in states:
        print('State',st)
        del X_test, y_test,g_test
        X_test, y_test,g_test=load_data(mode='test',year='2021',state_list=[st])
        with torch.no_grad():
                #pred0=l2g_fmt(torch.tensor(X_test).to(dv).float())
                #print('Epoch: ',j, 'Possible groups:', gps, 'at Layer', out)
                for t in range(N_tasks): 
                    SL.load_state_dict(torch.load(spaths['path'+str(t)]))
                    pred0=SL(torch.tensor(X_test).to(dv).float())
                    accuracy=acc(pred0.to(cpu),torch.tensor(y_test[t]))
                    EO=DM_rate(pred0.to(cpu),torch.tensor(y_test[t]).to(cpu),torch.tensor(g_test).to(cpu)) 
                    test_accuracy[t].append(accuracy)
                    test_fairness[t].append(EO)
                    print('Task',tasks[t],'  Acc:',accuracy, 'EO:',EO)
                    
    return test_accuracy,test_fairness

In [61]:
shapes=[1024,128,256,512,128]

In [62]:
Sls=trainstl_pums(shapes=shapes,year=2018)

In [63]:
test_acc,test_fair=test_pums(shapes=shapes,year='2019')

State CA
Task Employment   Acc: tensor(0.9986) EO: (tensor(0.0003), tensor(9.3520e-05))
Task Income   Acc: tensor(0.7601) EO: (tensor(0.0037), tensor(0.0009))
Task HealthInsurance   Acc: tensor(0.8682) EO: (tensor(0.0022), tensor(0.0021))
Task TravelTime   Acc: tensor(0.7027) EO: (tensor(0.0031), tensor(0.0019))
Task IncomePovertyRatio   Acc: tensor(0.7802) EO: (tensor(0.0045), tensor(0.0031))
State FL
Task Employment   Acc: tensor(0.9938) EO: (tensor(0.0007), tensor(0.0003))
Task Income   Acc: tensor(0.7482) EO: (tensor(0.0023), tensor(0.0014))
Task HealthInsurance   Acc: tensor(0.8171) EO: (tensor(0.0092), tensor(0.0075))
Task TravelTime   Acc: tensor(0.6865) EO: (tensor(0.0013), tensor(3.0756e-05))
Task IncomePovertyRatio   Acc: tensor(0.7403) EO: (tensor(0.0014), tensor(0.0007))
State GA
Task Employment   Acc: tensor(0.8297) EO: (tensor(0.0589), tensor(0.0109))
Task Income   Acc: tensor(0.7491) EO: (tensor(0.0235), tensor(0.0145))
Task HealthInsurance   Acc: tensor(0.8708) EO: (ten

Task Employment   Acc: tensor(0.7955) EO: (tensor(0.0342), tensor(0.0050))
Task Income   Acc: tensor(0.7303) EO: (tensor(0.0023), tensor(0.0003))
Task HealthInsurance   Acc: tensor(0.8771) EO: (tensor(0.0350), tensor(0.0295))
Task TravelTime   Acc: tensor(0.6326) EO: (tensor(0.0061), tensor(0.0041))
Task IncomePovertyRatio   Acc: tensor(0.7586) EO: (tensor(0.0282), tensor(0.0217))
State PA
Task Employment   Acc: tensor(0.8221) EO: (tensor(0.0047), tensor(0.0027))
Task Income   Acc: tensor(0.7251) EO: (tensor(0.0016), tensor(0.0004))
Task HealthInsurance   Acc: tensor(0.8729) EO: (tensor(0.0041), tensor(0.0027))
Task TravelTime   Acc: tensor(0.6521) EO: (tensor(0.0037), tensor(0.0025))
Task IncomePovertyRatio   Acc: tensor(0.7770) EO: (tensor(0.0040), tensor(0.0006))
State TX
Task Employment   Acc: tensor(0.9330) EO: (tensor(0.0648), tensor(0.0155))
Task Income   Acc: tensor(0.7584) EO: (tensor(0.0025), tensor(0.0010))
Task HealthInsurance   Acc: tensor(0.8789) EO: (tensor(0.0084), tens

In [64]:
best_acc=[sum(test_acc[t])/len(test_acc[t]) for t in range(N_tasks)]

In [20]:
year='2019'
torch.save(torch.tensor(best_acc),'/yourpath/Grouping_FMTL/'+year[-2:]+'acc.pt')
torch.save(torch.tensor(SEO),'/yourpath/Grouping_FMTL/'+year[-2:]+'seo.pt')
torch.save(torch.tensor(SEP),'/home/roy/Grouping_FMTL/'+year[-2:]+'sep.pt')